# Setup

In [ ]:
%%capture
!pip install efficientnet

In [ ]:
import re
import os
import math
import random
import itertools
import numpy as np
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
import matplotlib.cm as cm
from functools import partial
import matplotlib.pyplot as plt
import efficientnet.keras as efn
from datetime import datetime, date
from tensorflow.keras import backend as K
from sklearn.metrics import roc_curve, auc
from kaggle_datasets import KaggleDatasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve, confusion_matrix, classification_report

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split

In [ ]:
# Global Settings
SEED = 1
DIM = 768
EPOCHS = 25
BATCH_SIZE = 64
NUM_CLASSES = 1
VERBOSE_LEVEL = 1
META_CONTRIBUTION = 0.1

LR_MAX = 1e-6
LR_MIN = 1e-8
LR_START = 1e-4

In [ ]:
tpu = None
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Device:', tpu.master())
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except:
    strategy = tf.distribute.get_strategy()
print('Number of replicas:', strategy.num_replicas_in_sync)
    
if tpu:
    #BATCH_SIZE = 128  # increase the batch size if we have a tpu
    USE_TENSORBOARD = False # Tensorboard does not work with tpu

# seed everything
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# get the current timestamp. This timestamp is used to save the model data with a unique name
now = datetime.now()
today = date.today()
current_time = now.strftime("%H:%M:%S")
timestamp = str(today) + "_" + str(current_time)

# environment settings
print("Tensorflow version " + tf.__version__)
AUTOTUNE = tf.data.AUTOTUNE

# Data Augmentation

In [ ]:
# Here we apply some manual augmentations that cannot be done with tf.image, 
# such as shearing, zooming and translation. Rotation can be done in tf.image but only in factors of 90 degrees, 
# so we do it manually instead.
# Source: https://www.kaggle.com/teyang/melanoma-detection-using-effnet-and-meta-data#5.-Train-and-Evaluate-Model
ROT_ = 180.0
SHR_ = 2
HZOOM_ = 8.0
WZOOM_ = 8.0
HSHIFT_ = 8.0
WSHIFT_ = 8.0


def get_mat(rotation, shear, height_zoom, width_zoom, height_shift, width_shift):
    # returns 3x3 transformmatrix which transforms indicies
        
    # CONVERT DEGREES TO RADIANS
    rotation = math.pi * rotation / 180.
    shear    = math.pi * shear    / 180.

    def get_3x3_mat(lst):
        return tf.reshape(tf.concat([lst],axis=0), [3,3])
    
    # ROTATION MATRIX
    c1   = tf.math.cos(rotation)
    s1   = tf.math.sin(rotation)
    one  = tf.constant([1],dtype='float32')
    zero = tf.constant([0],dtype='float32')
    
    rotation_matrix = get_3x3_mat([c1,   s1,   zero, 
                                   -s1,  c1,   zero, 
                                   zero, zero, one])    
    # SHEAR MATRIX
    c2 = tf.math.cos(shear)
    s2 = tf.math.sin(shear)    
    
    shear_matrix = get_3x3_mat([one,  s2,   zero, 
                                zero, c2,   zero, 
                                zero, zero, one])        
    # ZOOM MATRIX
    zoom_matrix = get_3x3_mat([one/height_zoom, zero,           zero, 
                               zero,            one/width_zoom, zero, 
                               zero,            zero,           one])    
    # SHIFT MATRIX
    shift_matrix = get_3x3_mat([one,  zero, height_shift, 
                                zero, one,  width_shift, 
                                zero, zero, one])
    
    return K.dot(K.dot(rotation_matrix, shear_matrix), 
                 K.dot(zoom_matrix,     shift_matrix))


def transform(image, DIM=DIM):    
    # input image - is one image of size [dim,dim,3] not a batch of [b,dim,dim,3]
    # output - image randomly rotated, sheared, zoomed, and shifted
    XDIM = DIM%2 #fix for size 331
    
    rot = ROT_ * tf.random.normal([1], dtype='float32')
    shr = SHR_ * tf.random.normal([1], dtype='float32') 
    h_zoom = 1.0 + tf.random.normal([1], dtype='float32') / HZOOM_
    w_zoom = 1.0 + tf.random.normal([1], dtype='float32') / WZOOM_
    h_shift = HSHIFT_ * tf.random.normal([1], dtype='float32') 
    w_shift = WSHIFT_ * tf.random.normal([1], dtype='float32') 

    # GET TRANSFORMATION MATRIX
    m = get_mat(rot,shr,h_zoom,w_zoom,h_shift,w_shift) 

    # LIST DESTINATION PIXEL INDICES
    x   = tf.repeat(tf.range(DIM//2, -DIM//2,-1), DIM)
    y   = tf.tile(tf.range(-DIM//2, DIM//2), [DIM])
    z   = tf.ones([DIM*DIM], dtype='int32')
    idx = tf.stack( [x,y,z] )
    
    # ROTATE DESTINATION PIXELS ONTO ORIGIN PIXELS
    idx2 = K.dot(m, tf.cast(idx, dtype='float32'))
    idx2 = K.cast(idx2, dtype='int32')
    idx2 = K.clip(idx2, -DIM//2+XDIM+1, DIM//2)
    
    # FIND ORIGIN PIXEL VALUES           
    idx3 = tf.stack([DIM//2-idx2[0,], DIM//2-1+idx2[1,]])
    d    = tf.gather_nd(image, tf.transpose(idx3))
        
    return tf.reshape(d,[DIM, DIM,3])

In [ ]:
def color(x):
    """Color augmentation
    Args:
        x: Image

    Returns:
        Augmented image
    """
    x = tf.image.random_hue(x, 0.05)
    x = tf.image.random_brightness(x, 0.1, seed=SEED)
    x = tf.image.random_contrast(x, 0.6, 1.4, seed=SEED)
    x = tf.image.random_saturation(x, 0.6, 1.4, seed=SEED)
    return x


def flip(x):
    """Flip augmentation
    Args:
        x: Image to flip

    Returns:
        Augmented image
    """
    x = tf.image.random_flip_left_right(x, seed=SEED)
    x = tf.image.random_flip_up_down(x, seed=SEED)
    return x

In [ ]:
def augment_image(image, augment=True):  
    augmentations = [color, flip, transform] 
    if augment:
        # Data augmentation
        for f in augmentations:
            if random.randint(1, 10) <= 5:
                image = f(image)     
                
    image = tf.image.central_crop(image, DIM*0.95 / DIM)
    image = tf.image.resize(image, [DIM, DIM])
    image = tf.reshape(image, [DIM, DIM, 3]) 
                
    return image

# Data Loading

In [ ]:
def count_data_items(filenames):
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

The malignant-v2-768x768 dataset includes all malginant images from 2020, 2019, 2018 and 2017 competition as well as data directly from ISIC.
* Source: https://www.kaggle.com/c/siim-isic-melanoma-classification/discussion/169139
* The first 15 TFRecords contain the malignant images from this years 2020 comp. There are 584 malignant images.
* The next 15 TFRecords contain 580 images downloaded from ISIC's online gallery and are not included in the competition.
* The next 15 even numbered TFRecords contain the malignant images from 2018 2017 comp data.
* The next 15 odd numbered TFRecords contain the malignant images from 2019 new portion comp data. 
* Note: The 2019 Data may decrease the model performance. More Infos here: https://www.kaggle.com/c/siim-isic-melanoma-classification/discussion/168028 

In [ ]:
GCS_PATH_MALIGNANT = KaggleDatasets().get_gcs_path('malignant-v2-768x768')
GCS_PATH_BENIGN = KaggleDatasets().get_gcs_path('benign-melanoma-tfrecords-full')
GCS_PATH_TEST = KaggleDatasets().get_gcs_path('melanoma-test')

MALIGNANT_FILES = tf.io.gfile.glob(GCS_PATH_MALIGNANT + '/*.tfrec')
BENIGN_FILES = tf.io.gfile.glob(GCS_PATH_BENIGN + '/*.tfrec')
TEST_FILES = tf.io.gfile.glob(GCS_PATH_TEST + '/*.tfrec')

print("MALIGNANT TF Records", len(MALIGNANT_FILES))
print("BENIGN TF Records", len(BENIGN_FILES))
print("TEST TF Records", len(TEST_FILES))

In [ ]:
DATA_FROM_2020 = MALIGNANT_FILES[0:15]
DATA_ISIC = MALIGNANT_FILES[15:30]
DATA_17_18_19 = MALIGNANT_FILES[30:60]

# Because 2019 may potentially decrease the model performance, we exlude it
DATA__17_18 = []
for i in range(0, len(DATA_17_18_19), 2):
    DATA__17_18.append(DATA_17_18_19[i])

print("DATA_FROM_2020", len(DATA_FROM_2020))  
print("DATA_ISIC", len(DATA_ISIC))  
print("DATA__17_18", len(DATA__17_18)) 

In [ ]:
# There are always 200 images in a benign record, but only arround 40 in a malignant record
# So for the validation datset, we use 5 malignant files from 2020 and 1 benign file => 395 validation images
# We try to get the same distribution of 1:10 in both sets
TRAINING_FILENAMES = sum([DATA_FROM_2020[0:10], DATA_ISIC, DATA__17_18], BENIGN_FILES[10:140])
VALIDATION_FILENAMES = sum([DATA_FROM_2020[10:15], BENIGN_FILES[0:10]], [])

random.shuffle(TRAINING_FILENAMES)
random.shuffle(VALIDATION_FILENAMES)

In [ ]:
TRAINING_IMAGES = count_data_items(TRAINING_FILENAMES)

print("Training images", TRAINING_IMAGES)
print("Validation images", count_data_items(VALIDATION_FILENAMES))
print(" ")
print("Benign images in train set", count_data_items(BENIGN_FILES[10:140]))
print("Malignant images in train set", count_data_items(sum([DATA_FROM_2020[0:10], DATA_ISIC, DATA__17_18], [])))

In [ ]:
# Get the class weights and the inital bias
benign_cases = count_data_items(BENIGN_FILES[10:140])
malignant_cases = count_data_items(sum([DATA_FROM_2020[0:10], DATA_ISIC, DATA__17_18], []))

initial_bias = np.log([malignant_cases/benign_cases])
weight_for_0 = (1 / benign_cases)*(TRAINING_IMAGES)/2.0 
weight_for_1 = (1 / malignant_cases)*(TRAINING_IMAGES)/2.0
class_weight = {0: weight_for_0, 1: weight_for_1}

print(" ")
print(class_weight)

In [ ]:
def decode_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.cast(image, tf.float32)
    return image

In [ ]:
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
resizing_layer = tf.keras.layers.experimental.preprocessing.Resizing(DIM, DIM)

In [ ]:
def read_tfrecord(example, labeled):
    tfrecord_format = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "target": tf.io.FixedLenFeature([], tf.int64)
    } if labeled else {
        "image": tf.io.FixedLenFeature([], tf.string),
        "image_name": tf.io.FixedLenFeature([], tf.string)
    }
    example = tf.io.parse_single_example(example, tfrecord_format)
    image = decode_image(example['image'])
    if labeled:
        label = tf.cast(example['target'], tf.int32)
        return image, label
    idnum = example['image_name']
    return image, idnum

In [ ]:
def load_dataset(filenames, labeled=True, ordered=False):
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False # disable order, increase speed
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTOTUNE) # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order) # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.cache() # cache ds for performance gains
    dataset = dataset.map(partial(read_tfrecord, labeled=labeled), num_parallel_calls=AUTOTUNE)
    dataset = dataset.map(lambda x, y: (normalization_layer(x), y), num_parallel_calls=AUTOTUNE) # normalize the image so the values are between 0 and 255
    dataset = dataset.map(lambda x, y: (resizing_layer(x), y), num_parallel_calls=AUTOTUNE) # resize the images to the same height and width

    # returns a dataset of (image, label) pairs if labeled=True or (image, id) pairs if labeled=False
    return dataset

In [ ]:
def get_training_dataset(files=TRAINING_FILENAMES, augment=True):
    dataset = load_dataset(files, labeled=True)
    if augment:
        dataset = dataset.map(lambda x, y: (augment_image(x, augment=augment), y), num_parallel_calls=AUTOTUNE)
    dataset = dataset.repeat()
    dataset = dataset.shuffle(TRAINING_IMAGES, reshuffle_each_iteration=True)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset

In [ ]:
def get_validation_dataset(files=VALIDATION_FILENAMES, ordered=False, repeat=False):
    dataset = load_dataset(files, labeled=True, ordered=ordered)
    if repeat:
        dataset = dataset.repeat()
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset

In [ ]:
def get_test_dataset(ordered=False):
    dataset = load_dataset(TEST_FILES, labeled=False, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset

# Data Validation

In [ ]:
def batch_to_numpy_images_and_labels(data):
    images, labels = data
    numpy_images = images.numpy()
    numpy_labels = labels.numpy()
    # if numpy_labels.dtype == object: # binary string in this case, these are image ID strings
    #    numpy_labels = [None for _ in enumerate(numpy_images)]
    # If no labels, only image IDs, return None for labels (this is the case for test data)
    return numpy_images, numpy_labels


def title_from_label_and_target(label, correct_label):
    CLASSES = [0, 1]
    if correct_label is None:
        return CLASSES[label], True
    correct = (label == correct_label)
    return "{} [{}{}{}]".format(CLASSES[label], 'OK' if correct else 'NO', u"\u2192" if not correct else '',
                                CLASSES[correct_label] if not correct else ''), correct


def display_one_image(image, title, subplot, red=False, titlesize=16):
    plt.subplot(*subplot)
    plt.axis('off')
    plt.imshow(image)
    if len(title) > 0:
        plt.title(title, fontsize=int(titlesize) if not red else int(titlesize/1.2),
                  color='red' if red else 'black', fontdict={'verticalalignment': 'center'}, pad=int(titlesize/1.5))
    return (subplot[0], subplot[1], subplot[2]+1)


def display_batch_of_images(databatch, predictions=None, unbatched=False):
    """This will work with:
    display_batch_of_images(images)
    display_batch_of_images(images, predictions)
    display_batch_of_images((images, labels))
    display_batch_of_images((images, labels), predictions)
    """
    # data
    images, labels = batch_to_numpy_images_and_labels(
        databatch) if not unbatched else databatch
    if labels is None:
        labels = [None for _ in enumerate(images)]

    # auto-squaring: this will drop data that does not fit into square or square-ish rectangle
    rows = int(math.sqrt(len(images)))
    cols = len(images)//rows

    # size and spacing
    FIGSIZE = 13.0
    SPACING = 0.1
    subplot = (rows, cols, 1)
    if rows < cols:
        plt.figure(figsize=(FIGSIZE, FIGSIZE/cols*rows))
    else:
        plt.figure(figsize=(FIGSIZE/rows*cols, FIGSIZE))

    # display
    for i, (image, label) in enumerate(zip(images[:rows*cols], labels[:rows*cols])):
        title = 'Benign' if label == 0 else 'Malignant'
        correct = True
        if predictions is not None:
            title, correct = title_from_label_and_target(predictions[i], label)
        # magic formula tested to work from 1x1 to 10x10 images
        dynamic_titlesize = FIGSIZE*SPACING/max(rows, cols)*40+3
        subplot = display_one_image(
            image, title, subplot, not correct, titlesize=dynamic_titlesize)

    # layout
    plt.tight_layout()
    if label is None and predictions is None:
        plt.subplots_adjust(wspace=0, hspace=0)
    else:
        plt.subplots_adjust(wspace=SPACING, hspace=SPACING)
    plt.show()


In [ ]:
example_dataset = get_training_dataset(augment=False)
example_dataset = example_dataset.unbatch().batch(15)

In [ ]:
example_batch = iter(example_dataset) 
image_batch, label_batch = next(example_batch)
display_batch_of_images((image_batch, label_batch))

In [ ]:
augmented_images = [augment_image(x, augment=True) for x in image_batch]
augmented_images = [np.clip(x, 0, 1) for x in augmented_images]
labels = [l.numpy() for l in label_batch]
display_batch_of_images((augmented_images, labels), unbatched=True)

In [ ]:
# images are in float32 format with normalized values
for i in range(10):
    image = image_batch[i]
    print("min:", np.min(image), " -  max:", np.max(image))

print(image.dtype)

# Build Model

In [ ]:
def reset_model():
    model = tf.keras.models.load_model("stage_0.h5")
    model.load_weights("stage_0.hdf5")
    return model

In [ ]:
def get_model_parameters(lr, epochs):
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    loss = tf.keras.losses.BinaryCrossentropy(label_smoothing = 0.05),
    metrics = [
        tf.keras.metrics.BinaryAccuracy(name='accuracy'),
        tf.keras.metrics.AUC(name='auc'),
    ]

    return loss, metrics, optimizer

In [ ]:
def compile_model(model):
    loss, metrics, optimizer = get_model_parameters(LR_START, EPOCHS)
    if tpu:
        model.compile(
            loss=loss,
            metrics=metrics,
            optimizer=optimizer,
            # Reduce python overhead, and maximize the performance of your TPU
            # Anything between 2 and `steps_per_epoch` could help here.
            steps_per_execution=5,
        )
    else:
        model.compile(
            loss=loss,
            metrics=metrics,
            optimizer=optimizer,
        )

    return model

In [ ]:
def build_model(output_bias=None):
    if output_bias is not None:
        output_bias = tf.keras.initializers.Constant(output_bias)

    base_model = tf.keras.applications.EfficientNetB6(
        include_top=False, 
        weights='imagenet', 
        input_shape=[DIM,DIM,3]
    )

    base_model.trainable = False
    model = tf.keras.models.Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(8, activation='relu'),
        tf.keras.layers.Dense(NUM_CLASSES, activation='sigmoid', bias_initializer=output_bias)
    ])

    return model

In [ ]:
# Clear the session - this helps when we are creating multiple models
K.clear_session()

# Creating the model in the strategy scope places the model on the TPU
with strategy.scope():
    model = build_model(output_bias=initial_bias)
    model = compile_model(model)

model.summary()

# Initial Model Training

In [ ]:
training_dataset = get_training_dataset(augment=True)

In [ ]:
history = model.fit(
    training_dataset,
    epochs=15,
    steps_per_epoch=30,
    class_weight=class_weight,
    verbose=VERBOSE_LEVEL
)

# Full Training

In [ ]:
from tensorflow.python.framework import ops
from tensorflow.python.ops import math_ops
from tensorflow.python.eager import context


def cyclic_learning_rate(global_step,
                         learning_rate=0.01,
                         max_lr=0.1,
                         step_size=20.,
                         gamma=0.99994,
                         mode='triangular',
                         name=None):
    if global_step is None:
        raise ValueError("global_step is required for cyclic_learning_rate.")

    learning_rate = ops.convert_to_tensor(
        learning_rate, name="learning_rate")

    dtype = learning_rate.dtype
    global_step = math_ops.cast(global_step, dtype)
    step_size = math_ops.cast(step_size, dtype)

    def cyclic_lr():
        """Helper to recompute learning rate; most helpful in eager-mode."""
        # computing: cycle = floor( 1 + global_step / ( 2 * step_size ) )
        double_step = math_ops.multiply(2., step_size)
        global_div_double_step = math_ops.divide(global_step, double_step)
        cycle = math_ops.floor(math_ops.add(1., global_div_double_step))
        # computing: x = abs( global_step / step_size – 2 * cycle + 1 )
        double_cycle = math_ops.multiply(2., cycle)
        global_div_step = math_ops.divide(global_step, step_size)
        tmp = math_ops.subtract(global_div_step, double_cycle)
        x = math_ops.abs(math_ops.add(1., tmp))
        # computing: clr = learning_rate + ( max_lr – learning_rate ) * max( 0, 1 - x )
        a1 = math_ops.maximum(0., math_ops.subtract(1., x))
        a2 = math_ops.subtract(max_lr, learning_rate)
        clr = math_ops.multiply(a1, a2)
        if mode == 'triangular2':
            clr = math_ops.divide(clr, math_ops.cast(math_ops.pow(2, math_ops.cast(
                cycle-1, tf.int32)), tf.float32))
        if mode == 'exp_range':
            clr = math_ops.multiply(math_ops.pow(gamma, global_step), clr)
        return math_ops.add(clr, learning_rate, name=name)

    if not context.executing_eagerly():
        cyclic_lr = cyclic_lr()

    return cyclic_lr


def get_lr_callback(mode, learning_rate, max_lr, step_size):
    def lrfn(epoch):
        return float(
            cyclic_learning_rate(
                epoch,
                mode=mode,
                learning_rate=learning_rate,
                max_lr=max_lr,
                step_size=step_size,
            )().numpy()
        )
    lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=2)
    return lr_callback


def plot_clr(mode, learning_rate, max_lr, step_size, epochs):
    rates = []
    for i in range(0, epochs):
        x = cyclic_learning_rate(
            i,
            mode=mode,
            learning_rate=learning_rate,
            max_lr=max_lr,
            step_size=step_size,
        )().numpy()
        rates.append(x)

    plt.xlabel('Iterations (epochs)')
    plt.ylabel('Learning rate')
    plt.plot(range(epochs), rates)
    

def display_training_curves(training, validation, title, subplot):
  ax = plt.subplot(subplot)
  ax.plot(training)
  ax.plot(validation)
  ax.set_title('model '+ title)
  ax.set_ylabel(title)
  ax.set_xlabel('epoch')
  ax.legend(['training', 'validation'])

In [ ]:
mode='triangular2'
step_size=2.
clr_callback = get_lr_callback(mode, LR_MIN, LR_MAX, step_size)
plot_clr(mode, LR_MIN, LR_MAX, step_size, EPOCHS)

callbacks = [clr_callback]

In [ ]:
K.clear_session()
with strategy.scope():
    model.trainable = True
    model = compile_model(model)    
    
model.summary()

In [ ]:
steps_per_epoch = 200
validation_steps_per_epoch = 40

training_dataset = get_training_dataset(augment=True)
validation_dataset = get_validation_dataset(repeat=True)
    
history = model.fit(
    training_dataset,
    epochs=EPOCHS,
    callbacks=callbacks,
    class_weight=class_weight,
    steps_per_epoch=steps_per_epoch,
    validation_data=validation_dataset,
    validation_steps=validation_steps_per_epoch,
    verbose=VERBOSE_LEVEL
)

In [ ]:
model.save_weights("model_weights.hdf5")
model.save("model.h5")

# Model Evaluation

In [ ]:
def plot_auc(t_y, p_y, timestamp):
    """ Helper function to plot the auc curve

    Parameters:
        t_y (array): True binary labels
        p_y (array): Target scores

    Returns:
        Null
    """
    fpr, tpr, thresholds = roc_curve(t_y, p_y, pos_label=1)
    fig, c_ax = plt.subplots(1, 1, figsize=(8, 8))
    c_ax.plot(fpr, tpr, label='%s (AUC:%0.2f)' % ('Target', auc(fpr, tpr)))
    c_ax.plot([0, 1], [0, 1], color='navy', lw=1, linestyle='--')
    c_ax.legend()
    c_ax.set_xlabel('False Positive Rate')
    c_ax.set_ylabel('True Positive Rate')
        
        
def plot_confusion_matrix(cm, labels, timestamp):
    """ Helper function to plot a confusion matrix

        Parameters:
            cm (confusion matrix)

        Returns:
            Null
    """
    plt.imshow(cm, interpolation='nearest')
    plt.title('Confusion Matrix')
    plt.colorbar()
    tick_marks = np.arange(len(labels))
    plt.xticks(tick_marks, labels, rotation=55)
    plt.yticks(tick_marks, labels)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], 'd'), horizontalalignment="center",
                 color="white" if cm[i, j] < thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    

def plot_precision_recall_curve(labels, predictions):
    """ Helper function to plot the precision recall curve

    Parameters:
        precision
        recall

    Returns:
        Null
    """   
    precision, recall, thresholds = precision_recall_curve(labels, predictions)
    fig, ax = plt.subplots(figsize=(8,8))
    ax.plot(precision, recall)
    ax.set_xlabel('Recall')
    ax.set_ylabel('Precision')
        

def plot_history(history, timestamp):
    """ Helper function to plot the history of a tensorflow model

        Parameters:
            history (history object): The history from a tf model
            timestamp (string): The timestamp of the function execution

        Returns:
            Null
    """
    f = plt.figure()
    f.set_figwidth(15)

    f.add_subplot(1, 2, 1)
    plt.plot(history['val_loss'], label='val loss')
    plt.plot(history['loss'], label='train loss')
    plt.legend()
    plt.title("Modell Loss")

    f.add_subplot(1, 2, 2)
    plt.plot(history['val_accuracy'], label='val accuracy')
    plt.plot(history['accuracy'], label='train accuracy')
    plt.legend()
    plt.title("Modell Accuracy")
        
        
def plot_metrics(history, timestamp):
    metrics = ['loss', 'accuracy', 'auc']
    plt.figure(figsize=(10, 8))
    for n, metric in enumerate(metrics):
        name = metric.replace("_", " ").capitalize()
        plt.subplot(3, 1, n+1)
        plt.plot(history.epoch, history.history[metric], label='Train')
        plt.plot(history.epoch,
                 history.history['val_'+metric], linestyle="--", label='Val')
        plt.xlabel('Epoch')
        plt.ylabel(name)
        plt.legend()

        
def calc_f1(prec, recall):
    """ Helper function to calculate the F1 Score

        Parameters:
            prec (int): precision
            recall (int): recall

        Returns:
            f1 score (int)
    """
    return 2*(prec*recall)/(prec+recall) if recall and prec else 0


def pred_to_binary(pred, threshold):
    """ Helper function turn the model predictions into a binary (0,1) format

    Parameters:
        pred (float): Model prediction

    Returns:
        binary prediction (int)
    """
    if pred < threshold:
        return 0
    else:
        return 1

    
def predict_on_dataset(model, dataset):
    print("start predicting ...")
    labels = []
    predictions = []

    for image_batch, label_batch in iter(dataset):
        labels.append(label_batch.numpy())
        batch_predictions = model.predict(image_batch)
        predictions.append(batch_predictions)

    # flatten the lists
    labels = [item for sublist in labels for item in sublist]
    predictions = [item[0] for sublist in predictions for item in sublist]
    return predictions, labels


def evaluate_model(model, dataset, history, timestamp):
    predictions, labels = predict_on_dataset(model, dataset)

    # calculate the precision, recall and the thresholds
    precision, recall, thresholds = precision_recall_curve(labels, predictions)

    # calculate the f1 score
    f1score = [calc_f1(precision[i], recall[i])
               for i in range(len(thresholds))]

    # get the index from the highest f1 score
    idx = np.argmax(f1score)

    # get the precision, recall, threshold and the f1score
    precision = round(precision[idx], 4)
    recall = round(recall[idx], 4)
    threshold = round(thresholds[idx], 4)
    f1score = round(f1score[idx], 4)

    print('Precision:', precision)
    print('Recall:', recall)
    print('Threshold:', threshold)
    print('F1 Score:', f1score)

    # create a confusion matrix
    y_pred_binary = [pred_to_binary(x, threshold) for x in predictions]
    cm = confusion_matrix(labels, y_pred_binary)

    cm_plot_label = ['benign', 'malignant']
    plot_confusion_matrix(cm, cm_plot_label, timestamp)
    
    print(" ")

    # plot model history
    plot_metrics(history, timestamp)
    
    print(" ")

    # plot auc curve
    plot_auc(labels, predictions, timestamp)
    
    print(" ")
    
    # plot precision recall curve
    plot_precision_recall_curve(labels, predictions)

    return predictions, labels, threshold

In [ ]:
example_validation_dataset = get_validation_dataset(repeat=False)
predictions, labels, threshold = evaluate_model(
    model=model, 
    dataset=example_validation_dataset, 
    history=history,
    timestamp=timestamp
)

In [ ]:
target_names = ['Benign', 'Malignant']
y_pred_binary = [pred_to_binary(x, threshold) for x in predictions]
print(classification_report(labels, y_pred_binary, target_names=target_names))

# Model Interpretation with Grad CAM
Visualize how parts of the image affects neural network's output by looking into the activation maps.
Source: https://keras.io/examples/vision/grad_cam & https://arxiv.org/abs/1610.02391

In [ ]:
# Get the CNN
efficientnet_model = False
for layer in model.layers:
    if layer.name == "efficientnet-b6":
        efficientnet_model = layer

In [ ]:
IMAGE_PATH = "../input/malignant-v2-768x768/jpeg768/ISIC_0000296.jpg"
img = tf.keras.preprocessing.image.load_img(IMAGE_PATH, target_size=(512, 512))
img = tf.keras.preprocessing.image.img_to_array(img) / 255
plt.imshow(img)
origin_img = img

In [ ]:
# Get the prediction for the image from the model
prediction = model.predict(np.expand_dims(img, axis=0))
binary_prediction = [0 if x < 0.5 else 1 for x in prediction]
print("Prediction: " + ("Benign" if binary_prediction == 0 else "Malignant"))

In [ ]:
# First, we create a model that maps the input image to the activations
# of the last conv layer as well as the output predictions
grad_model = tf.keras.models.Model(
    [efficientnet_model.inputs], [efficientnet_model.get_layer('top_conv').output, efficientnet_model.output]
)

# Then, we compute the gradient of the top predicted class for our input image
# with respect to the activations of the last conv layer
with tf.GradientTape() as tape:
    last_conv_layer_output, preds = grad_model(np.expand_dims(img, axis=0))
    class_channel = preds[:, round(np.mean(tf.argmax(preds[0]).numpy()))]

# This is the gradient of the output neuron (top predicted or chosen)
# with regard to the output feature map of the last conv layer
grads = tape.gradient(class_channel, last_conv_layer_output)

# This is a vector where each entry is the mean intensity of the gradient
# over a specific feature map channel
pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

# We multiply each channel in the feature map array
# by "how important this channel is" with regard to the top predicted class
# then sum all the channels to obtain the heatmap class activation
last_conv_layer_output = last_conv_layer_output[0]
heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
heatmap = tf.squeeze(heatmap)

# For visualization purpose, we will also normalize the heatmap between 0 & 1
heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
heatmap = heatmap.numpy()

In [ ]:
# Display heatmap
plt.matshow(heatmap)
plt.show()

In [ ]:
# Load the original image
o_img = tf.keras.preprocessing.image.load_img(IMAGE_PATH, target_size=(512, 512))
o_img = tf.keras.preprocessing.image.img_to_array(img) * 255

# Rescale heatmap to a range 0-255
heatmap = np.uint8(255 * heatmap)

# Use jet colormap to colorize heatmap
jet = cm.get_cmap("jet")

# Use RGB values of the colormap
jet_colors = jet(np.arange(256))[:, :3]
jet_heatmap = jet_colors[heatmap]

# Create an image with RGB colorized heatmap
jet_heatmap = tf.keras.preprocessing.image.array_to_img(jet_heatmap)
jet_heatmap = jet_heatmap.resize((512,512))
jet_heatmap = tf.keras.preprocessing.image.img_to_array(jet_heatmap)

# Superimpose the heatmap on original image
superimposed_img = jet_heatmap * 0.4 + o_img
superimposed_img = tf.keras.preprocessing.image.array_to_img(superimposed_img)

In [ ]:
fig = plt.figure(figsize = (12, 8))

ax1 = fig.add_subplot(1, 2, 1)
ax1.imshow(superimposed_img)

ax2 = fig.add_subplot(1, 2, 2)
ax2.imshow(origin_img)

# Re-Training on misclassified images

In [ ]:
# Look at a handful of images and analyze them
NUM_IMAGES = count_data_items(TRAINING_FILENAMES)
print("NUM_IMAGES", NUM_IMAGES)

misclassified_analysis_ds = get_validation_dataset(files=TRAINING_FILENAMES, repeat=False, ordered=True)
misclassified_analysis_ds = misclassified_analysis_ds.unbatch().take(NUM_IMAGES).batch(BATCH_SIZE)

In [ ]:
misclassified_images = []
misclassified_predictions = []
misclassified_images_true_labels = []

for i, (img_batch, label_batch) in enumerate(tqdm(misclassified_analysis_ds, total=round(NUM_IMAGES / BATCH_SIZE))):
    predictions = clf.predict(img_batch, verbose=0, use_multiprocessing=True)
    binary_predictions = [0 if x < threshold else 1 for x in predictions]

    labels = label_batch.numpy()

    for i, label in enumerate(labels):
        if label != binary_predictions[i]:
            misclassified_images_true_labels.append(label)
            misclassified_predictions.append(predictions[i])
            misclassified_images.append(img_batch[i].numpy())

In [ ]:
print("Total misclassified", len(misclassified_images_true_labels) )
print("Misclassified malignant images", sum(misclassified_images_true_labels))

In [ ]:
img = misclassified_images[0]
true_label = misclassified_images_true_labels[0]
print("True Label: " + ("Benign" if true_label == 0 else "Malignant"))
plt.imshow(img)

In [ ]:
predicted_label = model.predict(tf.expand_dims(img, axis=0))
binary_prediction = 0 if predicted_label[0][0] < 0.5 else 1
print("Predicted Label: " + ("Benign" if binary_prediction == 0 else "Malignant"))

In [ ]:
x = [x[0] for x in misclassified_predictions]
print("Max", round(np.max(x), 4))
print("Min", round(np.min(x), 4))
print("Median", round(np.median(x), 4))
print("Total Misclassified", len(x))
print(" ")
_, ax1 = plt.subplots()
ax1.set_title('Distribution of Misclassified Images Predictions')
_ = ax1.boxplot(x, vert=False,)

In [ ]:
# Create a new augmented ds with 5 times the amount of miclassified images
misclassified_images_augmented = []
misclassified_images_true_labels_augmented = []

total = len(misclassified_images_true_labels) * 5
pbar = tqdm(total=total)

counter = 0
while len(misclassified_images_augmented) < total:
    if counter >= len(misclassified_images):
        counter = 0

    img = misclassified_images[counter]
    lbl = misclassified_images_true_labels[counter]
    img_aug = augment_image(img)

    misclassified_images_augmented.append(img_aug)
    misclassified_images_true_labels_augmented.append(lbl)

    counter = counter + 1
    pbar.update(1)

pbar.close()

In [ ]:
aug_img = misclassified_images_augmented[0]
print("Label:", misclassified_images_true_labels_augmented[0])
plt.imshow(aug_img)

In [ ]:
misclassified_ds = tf.data.Dataset.from_tensor_slices((misclassified_images_augmented, misclassified_images_true_labels_augmented))

training_dataset = get_training_dataset(augment=True)
training_dataset = training_dataset.unbatch().take(15000)

validation_dataset = get_validation_dataset(repeat=True)

In [ ]:
combined_ds = misclassified_ds.concatenate(training_dataset)
combined_ds = combined_ds.cache()
combined_ds = combined_ds.repeat()
combined_ds = combined_ds.shuffle(2048, seed=SEED, reshuffle_each_iteration=True)
combined_ds = combined_ds.batch(BATCH_SIZE)
combined_ds = combined_ds.prefetch(AUTOTUNE)

In [ ]:
lr_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-8)
callbacks = [lr_callback]

In [ ]:
steps_per_epoch = 200
validation_steps_per_epoch = 40
    
history = model.fit(
    combined_ds,
    epochs=EPOCHS,
    callbacks=callbacks,
    class_weight=class_weight,
    steps_per_epoch=steps_per_epoch,
    validation_data=validation_dataset,
    validation_steps=validation_steps_per_epoch,
    verbose=1
)

In [ ]:
example_validation_dataset = get_validation_dataset(repeat=False)
predictions, labels, threshold = evaluate_model(
    model=model, 
    dataset=example_validation_dataset, 
    history=history,
    timestamp=timestamp
)

y_pred_binary = [pred_to_binary(x, threshold) for x in predictions]
print(classification_report(labels, y_pred_binary, target_names=target_names))

# Meta Learner

In [ ]:
df = pd.read_csv("../input/siim-isic-melanoma-classification/train.csv")
df_test = pd.read_csv("../input/siim-isic-melanoma-classification/test.csv")

df.head()

In [ ]:
# getting dummy variables for gender
sex_dummies = pd.get_dummies(df['sex'], prefix='sex', dtype="int")
df = pd.concat([df, sex_dummies], axis=1)

# getting dummy variables for anatom_site_general_challenge
anatom_dummies = pd.get_dummies(df['anatom_site_general_challenge'], prefix='anatom', dtype="int")
df = pd.concat([df, anatom_dummies], axis=1)

# dropping not useful columns
df.drop(['sex','diagnosis','benign_malignant','anatom_site_general_challenge', 'image_name', 'patient_id'], axis=1, inplace=True)

# replace missing age values wiht the mean age
df['age_approx'] = df['age_approx'].fillna(int(np.mean(df['age_approx'])))

# convert age to int
df['age_approx'] = df['age_approx'].astype('int')

# Scale age column
scaler = StandardScaler()
df[['age_approx']] = scaler.fit_transform(df[['age_approx']])

df.head(3)

In [ ]:
feature_columns = ['age_approx', 'sex_female', 'sex_male', 'anatom_head/neck',
       'anatom_lower extremity', 'anatom_oral/genital', 'anatom_palms/soles',
       'anatom_torso', 'anatom_upper extremity']

target_columns = ['target']

df_train, df_test = train_test_split(df, test_size=0.20, random_state=SEED)

x_train = df_train[feature_columns]
y_train = df_train[target_columns]

x_test = df_test[feature_columns]
y_test = df_test[target_columns]

In [ ]:
# Source: https://www.kaggle.com/teyang/melanoma-detection-using-effnet-and-meta-data
meta_model = RandomForestClassifier(
    n_estimators=5000, 
    max_depth=5, 
    class_weight='balanced',
    n_jobs=-1, 
    random_state=SEED)

In [ ]:
fold_no = 1
kf = StratifiedKFold(5, shuffle=True, random_state=SEED)

for train_indexes, test_index in kf.split(x_train, y_train):    
    
    x_train_fold = x_train.iloc[train_indexes]
    y_train_fold = list(y_train.iloc[train_indexes].loc[:,'target'])
    y_train_fold = [int(x) for x in y_train_fold]

    x_test_fold = x_train.iloc[test_index]
    y_test_fold = list(y_train.iloc[test_index].loc[:,'target'])
    y_test_fold = [int(x) for x in y_test_fold]


    meta_model.fit(x_train_fold, y_train_fold)
    predictions = meta_model.predict(x_test_fold)

    print('Fold',str(fold_no), 'roc_auc_score:', roc_auc_score(y_test_fold, predictions))
    fold_no += 1

In [ ]:
# Prepare test df
df = pd.read_csv("../input/siim-isic-melanoma-classification/test.csv")
# getting dummy variables for gender
sex_dummies = pd.get_dummies(df['sex'], prefix='sex', dtype="int")
df = pd.concat([df, sex_dummies], axis=1)

# getting dummy variables for anatom_site_general_challenge
anatom_dummies = pd.get_dummies(df['anatom_site_general_challenge'], prefix='anatom', dtype="int")
df = pd.concat([df, anatom_dummies], axis=1)

# dropping not useful columns
df.drop(['sex','anatom_site_general_challenge', 'image_name', 'patient_id'], axis=1, inplace=True)

# replace missing age values wiht the mean age
df['age_approx'] = df['age_approx'].fillna(int(np.mean(df['age_approx'])))

# convert age to int
df['age_approx'] = df['age_approx'].astype('int')

# Scale age column
scaler = StandardScaler()
df[['age_approx']] = scaler.fit_transform(df[['age_approx']])


df_test = df[feature_columns]
meta_model_predictions = meta_model.predict_proba(df_test)
meta_model_predictions[0]

# Submission

In [ ]:
test_ds = get_test_dataset(ordered=True)
NUM_TEST_IMAGES = count_data_items(TEST_FILES)
test_ids_ds = test_ds.map(lambda image, idnum: idnum).unbatch()
test_ids = next(iter(test_ids_ds.batch(NUM_TEST_IMAGES))).numpy().astype('U')

In [ ]:
print('Computing predictions...')
test_images_ds = test_ds.map(lambda image, idnum: image)
probabilities = model.predict(test_images_ds, steps=math.ceil(len(test_ids) / BATCH_SIZE))

In [ ]:
print('Generating submission.csv file...')
test_ids_ds = test_ds.map(lambda image, idnum: idnum).unbatch()
test_ids = next(iter(test_ids_ds.batch(NUM_TEST_IMAGES))).numpy().astype('U')

pred_df = pd.DataFrame({'image_name': test_ids, 'target': np.concatenate(probabilities)})
pred_df.head()

In [ ]:
plt.hist(pred_df.target,bins=25)
plt.show()

In [ ]:
combined_predictions = []
df = pd.read_csv("../input/siim-isic-melanoma-classification/test.csv")

for index, row in pred_df.iterrows():
    image_name = row['image_name']
    target = row['target']
    idx_meta_prediction = df.index[df['image_name'] == image_name].values[0]    
    meta_prediction = meta_model_predictions[idx_meta_prediction][1]

    combined_prediction = float(target) + (meta_prediction * META_CONTRIBUTION)
    combined_predictions.append(combined_prediction)

pred_df_combined = pd.DataFrame({'image_name': test_ids, 'target': combined_predictions})
pred_df_combined.head()

In [ ]:
plt.hist(pred_df_combined.target,bins=25)
plt.show()

In [ ]:
pred_df_combined.to_csv("./submission.csv", index=False)